In [19]:
# lscp_mclp_pulp.py
from collections import defaultdict

import pandas as pd
import pulp

In [53]:
OD_CSV = 'raw_data/shelter_victim_matrix_walking_V8.csv'
S = 2                         # coverage threshold in minutes
# epsilon-constraint targets (fractions)
COVERAGE_TARGETS = [0.5, 0.6, 0.70, 0.80, 0.90, 0.95, 1.00]
# max p to sweep for MCLP (choose <= number of candidate sites)
PMAX = 30

In [54]:
# Read OD matrix
df = pd.read_csv(OD_CSV)

# Basic checks
required_cols = {'FROM_ID', 'TO_ID', 'DURATION_H', 'POP'}
if not required_cols.issubset(df.columns):
    raise SystemExit(f"Input CSV must contain columns: {required_cols}")

# Build sets and params
demands = sorted(df['FROM_ID'].unique().tolist())
sites = sorted(df['TO_ID'].unique().tolist())

# Map demand -> population (assume consistent)
pop = df[['FROM_ID', 'POP']].drop_duplicates().set_index('FROM_ID')[
    'POP'].to_dict()
total_pop = sum(pop.values())

# Build d_ij dict and coverage boolean
d_ij = {(row.FROM_ID, row.TO_ID): row.DURATION_H for row in df.itertuples()}
coverable = {(i, j): (d_ij[(i, j)] <= S) for i in demands for j in sites}

In [55]:
# Helper: pretty join

def sites_to_str(sol_sites):
    return ';'.join(sorted(sol_sites))

In [56]:
# ---------- LSCP (full coverage) ----------
prob = pulp.LpProblem("LSCP_full_coverage", pulp.LpMinimize)
y = pulp.LpVariable.dicts('y', sites, lowBound=0, upBound=1, cat='Binary')
# Objective: minimize number of sites
prob += pulp.lpSum(y[j] for j in sites)
# Constraints: each demand must be covered by at least one chosen site that is within S
for i in demands:
    # sites that can cover i
    Sj = [j for j in sites if coverable[(i, j)]]
    if not Sj:
        # This demand cannot be covered by any site within S -> LSCP infeasible
        prob = None
        break
    prob += pulp.lpSum(y[j] for j in Sj) >= 1, f"cover_{i}"

if prob is None:
    print(
        f"LSCP infeasible: some demand points cannot be covered within S={S} minutes by any candidate site.")
    lscp_result = None
else:
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    print(prob.status)
    selected = [j for j in sites if pulp.value(y[j]) > 0.5]
    lscp_result = {'num_sites': len(selected), 'selected_sites': selected}
    print("LSCP solved. Num sites:", lscp_result['num_sites'])

# Save LSCP results
if lscp_result:
    print(lscp_result)
    # pd.DataFrame([{'selected_sites': sites_to_str(lscp_result['selected_sites']),
    #              'num_sites': lscp_result['num_sites']}]).to_csv('lscp_solution.csv', index=False)

1
LSCP solved. Num sites: 11
{'num_sites': 11, 'selected_sites': ['sta_ndakwela_A1_3299', 'ta_chapananga_A1_3922', 'ta_chapananga_A2_3374', 'ta_makhwira_A1_5016', 'ta_makhwira_A2_5109', 'ta_maseya_A3_5485', 'ta_mlilima_A2_1818', 'ta_ngabu_A1_2294', 'ta_ngabu_A2_2745', 'ta_ngabu_A3_2056', 'ta_ngowe_A1_2979']}


In [35]:
# ---------- Epsilon-constraint: minimize sites subject to coverage >= target ----------
epsilon_rows = []
for target in COVERAGE_TARGETS:
    prob = pulp.LpProblem(f"LSCP_epsilon_{int(target*100)}", pulp.LpMinimize)
    y = pulp.LpVariable.dicts('y', sites, lowBound=0, upBound=1, cat='Binary')
    # x_i = 1 if demand i covered (by chosen site)
    x = pulp.LpVariable.dicts(
        'x', demands, lowBound=0, upBound=1, cat='Binary')
    # Objective
    prob += pulp.lpSum(y[j] for j in sites)
    # Link x and y
    for i in demands:
        Sj = [j for j in sites if coverable[(i, j)]]
        if Sj:
            prob += x[i] <= pulp.lpSum(y[j] for j in Sj)
        else:
            # Can't be covered at all, force x[i]=0
            prob += x[i] == 0
    # Coverage constraint
    prob += pulp.lpSum(pop[i] * x[i] for i in demands) >= target * total_pop
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    selected = [j for j in sites if pulp.value(y[j]) > 0.5]
    covered_demands = [i for i in demands if pulp.value(x[i]) > 0.5]
    covered_pop = sum(pop[i] for i in covered_demands)
    epsilon_rows.append({
        'coverage_target': target,
        'num_sites': len(selected),
        'selected_sites': sites_to_str(selected),
        'covered_pop': covered_pop,
        'coverage_pct': covered_pop/total_pop
    })
    print(f"Target {target:.2f}: num_sites={len(selected)}, coverage={covered_pop}/{total_pop} ({100*covered_pop/total_pop:.1f}%)")

pd.DataFrame(epsilon_rows).to_csv('epsilon_solutions.csv', index=False)

Target 0.50: num_sites=6, coverage=12656/24941 (50.7%)
Target 0.60: num_sites=8, coverage=15369/24941 (61.6%)
Target 0.70: num_sites=10, coverage=17567/24941 (70.4%)
Target 0.80: num_sites=14, coverage=20137/24941 (80.7%)
Target 0.90: num_sites=20, coverage=22474/24941 (90.1%)
Target 0.95: num_sites=18, coverage=22951/24941 (92.0%)
Target 1.00: num_sites=7, coverage=23780/24941 (95.3%)


In [40]:
# ---------- MCLP sweep (for p = 1..PMAX) ----------
mclp_rows = []
Pmax = min(PMAX, len(sites))
for p_val in range(1, Pmax+1):
    prob = pulp.LpProblem(f"MCLP_p_{p_val}", pulp.LpMaximize)
    y = pulp.LpVariable.dicts('y', sites, lowBound=0, upBound=1, cat='Binary')
    # 1 if demand covered by chosen sites
    x = pulp.LpVariable.dicts(
        'x', demands, lowBound=0, upBound=1, cat='Binary')
    # Objective: maximize covered population
    prob += pulp.lpSum(pop[i] * x[i] for i in demands)
    # Link x and y
    for i in demands:
        Sj = [j for j in sites if coverable[(i, j)]]
        if Sj:
            prob += x[i] <= pulp.lpSum(y[j] for j in Sj)
        else:
            prob += x[i] == 0
    # Exactly p sites chosen
    prob += pulp.lpSum(y[j] for j in sites) == p_val
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    selected = [j for j in sites if pulp.value(y[j]) > 0.5]
    covered_demands = [i for i in demands if pulp.value(x[i]) > 0.5]
    covered_pop = sum(pop[i] for i in covered_demands)
    mclp_rows.append({
        'p': p_val,
        'num_selected': len(selected),
        'selected_sites': sites_to_str(selected),
        'covered_pop': covered_pop,
        'coverage_pct': covered_pop/total_pop
    })
    print(f"p={p_val}: coverage {covered_pop}/{total_pop} ({100*covered_pop/total_pop:.1f}%) with sites: {selected}")

pd.DataFrame(mclp_rows).to_csv('mclp_sweep.csv', index=False)

print("Done. Outputs: lscp_solution.csv (if any), epsilon_solutions.csv, mclp_sweep.csv")

p=1: coverage 3685/24941 (14.8%) with sites: ['ta_ngabu_A1_2203']
p=2: coverage 5853/24941 (23.5%) with sites: ['ta_mlilima_A3_1884', 'ta_ngabu_A1_2203']
p=3: coverage 7836/24941 (31.4%) with sites: ['ta_mlilima_A3_1884', 'ta_ngabu_A1_2203', 'ta_ngabu_A3_2071']
p=4: coverage 9810/24941 (39.3%) with sites: ['ta_mlilima_A1_1879', 'ta_mlilima_A3_1884', 'ta_ngabu_A1_2203', 'ta_ngabu_A3_2071']
p=5: coverage 11695/24941 (46.9%) with sites: ['ta_mlilima_A1_1879', 'ta_mlilima_A3_1884', 'ta_ngabu_A1_2203', 'ta_ngabu_A2_2641', 'ta_ngabu_A3_2071']
p=6: coverage 13341/24941 (53.5%) with sites: ['sta_ndakwela_A1_3218', 'ta_mlilima_A1_1879', 'ta_mlilima_A3_1884', 'ta_ngabu_A1_2203', 'ta_ngabu_A2_2641', 'ta_ngabu_A3_2071']
p=7: coverage 14721/24941 (59.0%) with sites: ['sta_ndakwela_A1_3218', 'ta_mlilima_A1_1879', 'ta_mlilima_A3_1884', 'ta_ngabu_A1_2203', 'ta_ngabu_A1_2361', 'ta_ngabu_A2_2641', 'ta_ngabu_A3_2071']
p=8: coverage 15912/24941 (63.8%) with sites: ['sta_ndakwela_A1_3218', 'ta_lundu_A2_477

In [62]:
import pulp

# --- Sample data ---
facilities = ["S1", "S2", "S3"]
villages = ["A", "B", "C", "D", "E"]

# travel_time[i][j] = time from village i to facility j (minutes)
travel_time = {
    "A": {"S1": 30, "S2": 70, "S3": 55},
    "B": {"S1": 25, "S2": 45, "S3": 75},
    "C": {"S1": 65, "S2": 40, "S3": 50},
    "D": {"S1": 80, "S2": 35, "S3": 45},
    "E": {"S1": 90, "S2": 60, "S3": 40},
}

# population at each demand point
population = {"A": 8000, "B": 6000, "C": 7000, "D": 10000, "E": 9000}

# parameters
max_travel_time = 60   # minutes
capacity_limit = 20000

# --- Model ---
model = pulp.LpProblem("Capacitated_LSCP", pulp.LpMinimize)

# decision variables
x = pulp.LpVariable.dicts("open_facility", facilities, cat="Binary")
y = pulp.LpVariable.dicts(
    "cover", [(i, j) for i in villages for j in facilities], cat="Binary")

# objective: minimize number of open facilities
model += pulp.lpSum(x[j] for j in facilities)

# coverage constraints (within time limit)
for i in villages:
    model += pulp.lpSum(y[i, j]
                        for j in facilities if travel_time[i][j] <= max_travel_time) <= 1

# linking coverage to open facilities
for i in villages:
    for j in facilities:
        if travel_time[i][j] <= max_travel_time:
            model += y[i, j] <= x[j]
        else:
            model += y[i, j] == 0

# capacity constraints (max 20,000 people per facility)
for j in facilities:
    model += pulp.lpSum(population[i] * y[i, j]
                        for i in villages) <= capacity_limit

# --- Solve ---
model.solve(pulp.PULP_CBC_CMD(msg=0))

# --- Results ---
results = {}
for j in facilities:
    if x[j].value() == 1:
        covered = [i for i in villages if y[i, j].value() == 1]
        total_pop = sum(population[i] for i in covered)
        results[j] = {"covered_demand": covered,
                      "population_served": total_pop}

print("Open facilities:", [f for f in facilities if x[f].value() == 1])
print("Facility coverage results:")
print(results)

Open facilities: []
Facility coverage results:
{}
